### Importing data gathered from twitter

In [1]:
import os
import sys
import glob

data_directory = os.path.join(os.path.dirname(os.getcwd()),'data/twitter')

files = glob.glob(os.path.join(data_directory,'*.csv'))
files

['/Users/sami/Projects/vote2017/data/twitter/fillon.csv',
 '/Users/sami/Projects/vote2017/data/twitter/hamon.csv',
 '/Users/sami/Projects/vote2017/data/twitter/lepen.csv',
 '/Users/sami/Projects/vote2017/data/twitter/macron.csv',
 '/Users/sami/Projects/vote2017/data/twitter/melenchon.csv',
 '/Users/sami/Projects/vote2017/data/twitter/valls.csv']

In [39]:
import pandas as pd
import csv
from gensim import corpora
from gensim.models.word2vec import LineSentence
sys.path.append(os.path.dirname(os.getcwd()))
from utils.preprocessing import preprocess_sentence

corpus = {}

for file in files:
    print('reading ',file)
    df = pd.read_csv(file, sep=';', quoting=csv.QUOTE_ALL)
    text = df.text.values
    tweets = []
    for t in text:
        tweets.append(preprocess_sentence(t))
    name = file.split('/')[-1]
    corpus[name] = tweets

('reading ', '/Users/sami/Projects/vote2017/data/twitter/fillon.csv')
('reading ', '/Users/sami/Projects/vote2017/data/twitter/hamon.csv')
('reading ', '/Users/sami/Projects/vote2017/data/twitter/lepen.csv')
('reading ', '/Users/sami/Projects/vote2017/data/twitter/macron.csv')
('reading ', '/Users/sami/Projects/vote2017/data/twitter/melenchon.csv')
('reading ', '/Users/sami/Projects/vote2017/data/twitter/valls.csv')
